In [1]:
import cv2
# import numpy as np
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox
import os

def augment(img_path,annotation_path,filename,annotation_folder_aug,aug_path):
    # Define the image and annotation file paths

    # Load the image and annotation file
    img = cv2.imread(img_path)
    with open(annotation_path, 'r') as f:
        annotation = f.readlines()

    # Define the augmentation pipeline
    # augmentation = iaa.Sequential([
    #     iaa.Sometimes(0.5, iaa.Affine(rotate=(-10, 10))),
    #     iaa.Sometimes(0.5, iaa.Affine(scale={"x": (1.1, 1.5), "y": (1.1, 1.5)})),
    #     iaa.Fliplr(0.5),
    #     iaa.Flipud(0.5),
    #     # iaa.Sometimes(0.5, iaa.SkewX((-30, 30))),
    #     # iaa.Sometimes(0.5, iaa.SkewY((-30, 30))),
    # ])
    augmentation = iaa.Sequential([
    iaa.Flipud(0.5),
    iaa.Fliplr(0.5),
    iaa.AdditiveGaussianNoise(scale=(0, 0.1*255)),
    iaa.MultiplyAndAddToBrightness(mul=(0.5, 1.5), add=(-30, 30)),
    iaa.MultiplyHueAndSaturation((0.5, 1.5), per_channel=True),
    iaa.SomeOf(2, [
        iaa.Add(50, per_channel=True),
        iaa.Sharpen(alpha=0.5),
        iaa.GaussianBlur(sigma=(0.0, 3.0))
        ])
    ])

    augmentation_flip_noise = iaa.Sequential([
    iaa.Flipud(0.5),
    iaa.Fliplr(0.5),
    iaa.AdditiveGaussianNoise(scale=(0, 0.1*255)),
    ])

    augmentation_fog = iaa.Sequential([
    iaa.Fog(2),
    iaa.Snowflakes((0.005, 0.075)),
    ])

    # augmentation_crop = iaa.Sequential([
    # iaa.SomeOf(2, [
    #     iaa.CropAndPad(percent=(-0.25, 0.25)),
    #     iaa.CropToAspectRatio(2.0)
    #     ])
    
    # ])

    


    # Extract the bounding box coordinates from the annotation file
    boxes = []
    for ann in annotation:
        class_id, x_center, y_center, width, height = map(float, ann.strip().split())
        left = int((x_center - width / 2) * img.shape[1])
        top = int((y_center - height / 2) * img.shape[0])
        right = int((x_center + width / 2) * img.shape[1])
        bottom = int((y_center + height / 2) * img.shape[0])
        boxes.append(BoundingBox(x1=left, y1=top, x2=right, y2=bottom))

    
    # Apply the augmentations to the image and update the bounding box coordinates
    img_aug, boxes_aug = augmentation_fog(image=img, bounding_boxes=boxes)

    # Convert the bounding box coordinates to YOLO format
    height, width, _ = img_aug.shape
    boxes_yolo = []
    for box in boxes_aug:
        x_center = (box.x1 + box.x2) / 2 / width
        y_center = (box.y1 + box.y2) / 2 / height
        box_width = (box.x2 - box.x1) / width
        box_height = (box.y2 - box.y1) / height
        class_id = 0
        boxes_yolo.append([class_id, x_center, y_center, box_width, box_height])

    aug_img_path = os.path.join(aug_path, "aug3_" + filename)
    aug_path = os.path.join(annotation_folder_aug, "aug3_" + filename[:-4] + ".txt")

    # Write the augmented image and annotation file
    cv2.imwrite(aug_img_path, img_aug)
    with open(aug_path, 'w') as f:
        for box in boxes_yolo:
            f.write('{} {} {} {} {}\n'.format(box[0], box[1], box[2], box[3], box[4]))


# Set the path to your image and annotation folders
image_folder = "/Users/erictommathews/Documents/Eric/Msc Artificial Intelligence - Stirling/Semester 3 Spring/Deep Learning/Assignment/ITNPAI1-SkyGrove/Dataset/Stirling/Resized"
annotation_folder = "/Users/erictommathews/Documents/Eric/Msc Artificial Intelligence - Stirling/Semester 3 Spring/Deep Learning/Assignment/ITNPAI1-SkyGrove/Dataset/Stirling/annotations"
aug_path = '/Users/erictommathews/Documents/Eric/Msc Artificial Intelligence - Stirling/Semester 3 Spring/Deep Learning/Assignment/ITNPAI1-SkyGrove/Dataset/Stirling/augmented'
annotation_folder_aug = '/Users/erictommathews/Documents/Eric/Msc Artificial Intelligence - Stirling/Semester 3 Spring/Deep Learning/Assignment/ITNPAI1-SkyGrove/Dataset/Stirling/aug_annotations'

for filename in os.listdir(image_folder):
    if filename.endswith(".jpg"):
        # Load the image
        image_path = os.path.join(image_folder, filename)
         #Load the corresponding annotation file
        annotation_path = os.path.join(annotation_folder, filename[:-4] + ".txt")
        augment(image_path,annotation_path,filename,annotation_folder_aug,aug_path)

In [ ]:
# import cv2
# %matplotlib inline
# #The line above is necesary to show Matplotlib's plots inside a Jupyter Notebook
# from matplotlib import pyplot as plt
# import imgaug.augmenters as iaa
# import numpy as np

# import os
# from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage

# # Set the path to your image and annotation folders
# image_folder = "/Users/erictommathews/Documents/Eric/Msc Artificial Intelligence - Stirling/Semester 3 Spring/Deep Learning/Assignment/ITNPAI1-SkyGrove/Dataset/Stirling/Resized"
# annotation_folder = "/Users/erictommathews/Documents/Eric/Msc Artificial Intelligence - Stirling/Semester 3 Spring/Deep Learning/Assignment/ITNPAI1-SkyGrove/Dataset/Stirling/annotations"
# aug_path = '/Users/erictommathews/Documents/Eric/Msc Artificial Intelligence - Stirling/Semester 3 Spring/Deep Learning/Assignment/ITNPAI1-SkyGrove/Dataset/Stirling/augmented'
# annotation_folder_aug = '/Users/erictommathews/Documents/Eric/Msc Artificial Intelligence - Stirling/Semester 3 Spring/Deep Learning/Assignment/ITNPAI1-SkyGrove/Dataset/Stirling/aug_annotations'

# # Define the augmentations you want to apply to your images
# augmentation_seq = iaa.Sequential([
#     iaa.Flipud(0.5),
#     iaa.Fliplr(0.5),
#     # iaa.Affine(rotate=(-45, 45)),
#     iaa.AdditiveGaussianNoise(scale=(0, 0.1*255))
# ])

# # def yolobbox2bbox(x,y,w,h):
# #     x1, y1 = x-w/2, y-h/2
# #     x2, y2 = x+w/2, y+h/2
# #     return x1, y1, x2, y2

# def convert_yolo_to_bbox(yolo_box, image_width, image_height):
#     class_index, x_center, y_center, width, height = map(float, yolo_box.split())

#     # Convert YOLO format to bbox format
#     x_min = int((x_center - width / 2) * image_height)
#     y_min = int((y_center - height / 2) * image_width)
#     x_max = int((x_center + width / 2) * image_height)
#     y_max = int((y_center + height / 2) * image_width)

#     # Return the bbox coordinates as a tuple
#     return (x_min, y_min, x_max, y_max)


# def convert_bbox_to_yolo(bbox, image_width, image_height, class_index):
#     # Calculate the center of the box
#     x_center = (bbox[0] + bbox[2]) / 2 / image_width
#     y_center = (bbox[1] + bbox[3]) / 2 / image_height

#     # Calculate the width and height of the box relative to the image size
#     width = (bbox[2] - bbox[0]) / image_width
#     height = (bbox[3] - bbox[1]) / image_height

#     # Format the output in YOLO format
#     yolo_output = f"{class_index} {x_center} {y_center} {width} {height}"
#     return yolo_output

# # Loop through the image and annotation folders and apply the augmentations to each image
# for filename in os.listdir(image_folder):
#     if filename.endswith(".jpg"):
#         # Load the image
#         image_path = os.path.join(image_folder, filename)
#         image = cv2.imread(image_path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         # Get the width and height of the image
#         image_width, image_height, _ = image.shape     
#         # Load the corresponding annotation file
#         annotation_path = os.path.join(annotation_folder, filename[:-4] + ".txt")
#         with open(annotation_path, 'r') as f:
#             annotation_data = [line.rstrip() for line in f]

#         # bounding_boxes_array = np.fromstring(annotation_data, sep=' ')
#         # bboxes = bounding_boxes_array[1:]
#         ls = []
#         for line in annotation_data:
#             # bounding_boxes_array = np.fromstring(line, sep=' ')
#             # print(line)
#             box = convert_yolo_to_bbox(line,image_width,image_height)
#             ls.append(BoundingBox(x1=box[0], y1=box[1], x2=box[2], y2=box[3]))

#         bbs = BoundingBoxesOnImage(ls, shape=image.shape)

#         # Apply the augmentations to the image and update the corresponding annotation file
#         augmented_image, augmented_annotation_data = augmentation_seq(image=image, bounding_boxes=bbs)
             
#             # image with BBs before/after augmentation (shown below)
#         image_before = bbs.draw_on_image(image, size=2)
#         image_after = augmented_annotation_data.draw_on_image(augmented_image, size=2, color=[0, 0, 255])
#         # Convert bbs_aug to a list of bounding boxes
#         bboxes_list = augmented_annotation_data.to_xyxy_array().tolist()

#         # Print the result
#         image_width, image_height, _ = augmented_image.shape
#         yolo_out = ""
#         for box in bboxes_list:
#             yolo_out += convert_bbox_to_yolo(box,image_width,image_height,0)+'\n'
#             # print(box)
#             print(yolo_out)
#         # Save the augmented image and annotation file
#         augmented_image_path = os.path.join(aug_path, "aug_" + filename)
#         cv2.imwrite(augmented_image_path, augmented_image)
#         augmented_annotation_path = os.path.join(annotation_folder_aug, "aug_" + filename[:-4] + ".txt")
#         with open(augmented_annotation_path, 'w') as f:
#             f.write(yolo_out)
